In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
from main_io_comp_vis_all import calibrate_magnetic_wifi_ibeacon_to_position, read_data_file
import warnings
warnings.filterwarnings('ignore')

In [ ]:
test_df = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')
test_df[["site","path","time"]]=test_df["site_path_timestamp"].str.split('_', expand=True)
site_list = list(set(list(test_df["site"])))
path_list = list(set(list(test_df["path"])))
len(path_list)

# 1) Format Test data

## 1.1) Initialize

In [ ]:
floor_txt_num = {'F1':1,'1F':1,'L1':1,'1L':1,'F2':3,'2F':3,'L2':3,'2L':3,'F3':4,'3F':4,'L3':4,'3L':4,'F4':5,'4F':5,'L4':5,'4L':5,
                'F5':6,'5F':6,'L5':6,'5L':6,'F6':7,'6F':7,'L6':7,'6L':7,'F7':8,'7F':8,'L7':8,'7L':8,'F8':9,'8F':9,'L8':9,'8L':9,
                'F9':10,'9F':10,'L9':10,'9L':10,'F10':11,'10F':11,'L10':11,'10L':11,'F11':12,'11F':12,'L11':12,'11L':12,
                'B1':1,'1B':1,"B":1,'B2':2,'2B':2,'B3':3,'3B':3,'B4':4,'4B':4}

In [ ]:
NUM_FEATS=10
BSSID_FEATS = [f'BSSID_{i}' for i in range(NUM_FEATS)]
RSSI_FEATS = [f'RSSI_{i}' for i in range(NUM_FEATS)]
BEACON_FEATS = [f'BEACONID_{i}' for i in range(NUM_FEATS)]
BEACONRSSI_FEATS = [f'BEACONRSSI_{i}' for i in range(NUM_FEATS)]
test_df[BSSID_FEATS] = "XXX"
test_df[BEACON_FEATS] = "XXX"
test_df[RSSI_FEATS] = -100
test_df[BEACONRSSI_FEATS] = -100

## 1.2) Generate Test data

In [ ]:
for idxp, path in enumerate(path_list):
    data = read_data_file('../input/indoor-location-navigation/test/'+path+'.txt')
    if (data.wifi != []):
        wifi_df = pd.DataFrame(data=data.wifi, columns= ["Time","SSID","BSSID","RSSI","LTime"])
        wifi_df = wifi_df.drop(columns=["SSID","LTime"])
        wifi_df[["Time", "RSSI"]] = wifi_df[["Time", "RSSI"]].astype('int')
    else:
        wifi_df = pd.DataFrame(data={"Time":[],"SSID":[],"BSSID":[],"RSSI":[],"LTime":[]})
    if (data.ibeacon != []):
        beacon_df = pd.DataFrame(data=data.ibeacon, columns= ["Time","BEACONID","RSSI"])
        beacon_df[["Time","RSSI"]] = beacon_df[["Time","RSSI"]].astype('int')
    else:
        beacon_df = pd.DataFrame(data={"Time":[],"BEACONID":[],"RSSI":[]})
    
    test_df_time = list(test_df.loc[test_df["path"]==path,["time"]].values)
    test_df_time = [int(i) for i in test_df_time]
    
    for time in test_df_time:
        wifi_df_temp = wifi_df.loc[(wifi_df["Time"]<(time+2000)) & (wifi_df["Time"]>(time-2000)),:]
        wifi_df_temp = wifi_df_temp.groupby(by=["BSSID"]).mean()
        wifi_df_temp = wifi_df_temp.sort_values(by=["RSSI"],ascending=False)
        wifi_df_temp = wifi_df_temp[:10]
        wifi_df_temp_bssid = list(wifi_df_temp.index)
        wifi_df_temp_rssi = list(wifi_df_temp["RSSI"])
        for i in range(10-len(wifi_df_temp)):
            wifi_df_temp_bssid.append('XXX')
            wifi_df_temp_rssi.append(-100)
        test_df.loc[(test_df["path"]==path) & (test_df["time"].astype('int')==time),BSSID_FEATS] = wifi_df_temp_bssid
        test_df.loc[(test_df["path"]==path) & (test_df["time"].astype('int')==time),RSSI_FEATS] = wifi_df_temp_rssi
        # print(wifi_df_temp_bssid)
        # print(test_df.loc[(test_df["path"]==path) & (test_df["time"].astype('int')==time),BSSID_FEATS])
        
        
        beacon_df_temp = beacon_df.loc[(beacon_df["Time"]<(time+2000)) & (beacon_df["Time"]>(time-2000)),:]
        beacon_df_temp = beacon_df_temp.groupby(by=["BEACONID"]).mean()
        beacon_df_temp = beacon_df_temp.sort_values(by=["RSSI"],ascending=False)
        beacon_df_temp = beacon_df_temp[:10]
        beacon_df_temp_beaconid = list(beacon_df_temp.index)
        beacon_df_temp_rssi = list(beacon_df_temp["RSSI"])
        for i in range(10-len(beacon_df_temp)):
            beacon_df_temp_beaconid.append('XXX')
            beacon_df_temp_rssi.append(-100)
        test_df.loc[(test_df["path"]==path) & (test_df["time"].astype('int')==time),BEACON_FEATS] = beacon_df_temp_beaconid
        test_df.loc[(test_df["path"]==path) & (test_df["time"].astype('int')==time),BEACONRSSI_FEATS] = beacon_df_temp_rssi
    print("FILE : ", idxp, " of 626")
        
test_df.to_csv('sub_file.csv')